## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from google.colab import files

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head(5)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop(columns=['EIN','NAME'])

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
type_counts = application_df["APPLICATION_TYPE"].value_counts()

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = type_counts[type_counts < 200].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_binning = application_df["CLASSIFICATION"].value_counts()
classification_binning

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_binning[classification_binning>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_binning[classification_binning<800].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   APPLICATION_TYPE_Other        34299 non-null  uint8
 4   APPLICATION_TYPE_T10          34299 non-null  uint8
 5   APPLICATION_TYPE_T19          34299 non-null  uint8
 6   APPLICATION_TYPE_T3           34299 non-null  uint8
 7   APPLICATION_TYPE_T4           34299 non-null  uint8
 8   APPLICATION_TYPE_T5           34299 non-null  uint8
 9   APPLICATION_TYPE_T6           34299 non-null  uint8
 10  APPLICATION_TYPE_T7           34299 non-null  uint8
 11  APPLICATION_TYPE_T8           34299 non-null  uint8
 12  AFFILIATION_CompanySponsored  34299 non-null  uint8
 13  AFFILIATION_Family/Parent     3

In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns = "IS_SUCCESSFUL")
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Input and first hidden layer with more units and Batch Normalization
nn.add(Dense(units=64, input_dim=43, activation="LeakyReLU"))
nn.add(BatchNormalization())
nn.add(Dropout(0.1))

# First hidden layer
nn.add(Dense(units=48, activation="LeakyReLU"))
nn.add(BatchNormalization())
nn.add(Dropout(0.1))

# Second hidden layer
nn.add(Dense(units=32, activation="LeakyReLU"))
nn.add(BatchNormalization())
nn.add(Dropout(0.1))

# Third hidden layer
nn.add(Dense(units=16, activation="LeakyReLU"))
nn.add(BatchNormalization())
nn.add(Dropout(0.1))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1 , activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_93 (Dense)            (None, 64)                2816      
                                                                 
 batch_normalization_28 (Ba  (None, 64)                256       
 tchNormalization)                                               
                                                                 
 dropout_24 (Dropout)        (None, 64)                0         
                                                                 
 dense_94 (Dense)            (None, 48)                3120      
                                                                 
 batch_normalization_29 (Ba  (None, 48)                192       
 tchNormalization)                                               
                                                                 
 dropout_25 (Dropout)        (None, 48)              

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=40)

Epoch 1/40
804/804 [==============================] - 6s 5ms/step - loss: 0.5991 - accuracy: 0.7012
Epoch 2/40
804/804 [==============================] - 3s 4ms/step - loss: 0.5794 - accuracy: 0.7180
Epoch 3/40
804/804 [==============================] - 3s 4ms/step - loss: 0.5725 - accuracy: 0.7201
Epoch 4/40
804/804 [==============================] - 3s 4ms/step - loss: 0.5670 - accuracy: 0.7252
Epoch 5/40
804/804 [==============================] - 4s 5ms/step - loss: 0.5656 - accuracy: 0.7242
Epoch 6/40
804/804 [==============================] - 4s 5ms/step - loss: 0.5645 - accuracy: 0.7262
Epoch 7/40
804/804 [==============================] - 3s 4ms/step - loss: 0.5635 - accuracy: 0.7273
Epoch 8/40
804/804 [==============================] - 4s 5ms/step - loss: 0.5644 - accuracy: 0.7246
Epoch 9/40
804/804 [==============================] - 3s 4ms/step - loss: 0.5624 - accuracy: 0.7230
Epoch 10/40
804/804 [==============================] - 3s 4ms/step - loss: 0.5613 - accuracy: 0.7254

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5520 - accuracy: 0.7310 - 737ms/epoch - 3ms/step
Loss: 0.5520430207252502, Accuracy: 0.7309620976448059


In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharityOptimized.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
 files.download('AlphabetSoupCharityOptimized.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>